In [ ]:
import requests
import json
from io import BytesIO
from PIL import Image
import numpy as np
import cv2

In [ ]:
def generate_equally_spaced_colors(k):
    colors = []
    step = 360 / k  # Equally spaced hue step

    for i in range(k):
        hue = i * step  # Equally spaced hue values
        rgb = hsv_to_rgb(hue, 1, 1)  # Convert hue to RGB values
        scaled_rgb = tuple(
            int(val * 255) for val in rgb
        )  # Scale RGB values to 0-255 range
        colors.append(scaled_rgb)

    return colors


def hsv_to_rgb(h, s, v):
    c = v * s
    x = c * (1 - abs((h / 60) % 2 - 1))
    m = v - c

    if 0 <= h < 60:
        rgb = (c, x, 0)
    elif 60 <= h < 120:
        rgb = (x, c, 0)
    elif 120 <= h < 180:
        rgb = (0, c, x)
    elif 180 <= h < 240:
        rgb = (0, x, c)
    elif 240 <= h < 300:
        rgb = (x, 0, c)
    else:
        rgb = (c, 0, x)

    return tuple((val + m) for val in rgb)

In [ ]:
with open("cat_dog.json") as f:
    data = json.load(f)

In [ ]:
##Get Images that already have segmentation masks
for image in data["images"]:
    if 'segmentation' in image['tags'][0]:
        print(image['id'])

In [ ]:
##Extract first image with segmentation mask
for image in data["images"]:
    if image['id'] == "64b7210228151f0007746978":
        break

In [ ]:
len(image['tags'][1]['segmentation'])

In [ ]:
##Get contours for image
ext_points = image['tags'][0]['segmentation'][0]['extPoints']
int_points = image['tags'][0]['segmentation'][0]['intPoints']

In [ ]:
image['url']

In [ ]:
req = requests.get("http://" + image["url"])
img = Image.open(BytesIO(req.content)).convert("RGB")

In [ ]:
img

In [ ]:
import numpy as np
from PIL import Image, ImageDraw

def generate_binary_mask(exterior_points, interior_points, image_size):
    # Create a blank image
    image = Image.new('L', image_size, 0)
    draw = ImageDraw.Draw(image)

    # Draw the exterior contour
    draw.polygon(exterior_points, outline=1, fill=1)

    # Draw the interior contours
    for interior_contour in interior_points:
        draw.polygon(interior_contour, outline=0, fill=0)

    # Convert the image to a binary mask (numpy array)
    binary_mask = np.array(image)

    return binary_mask


In [ ]:
img.size

In [ ]:
##Format countours in right format
int_points = [list(map(tuple,int)) for int in int_points]

In [ ]:
##Format countours in right format
ext_points = list(map(tuple,ext_points))

In [ ]:
cat_mask = generate_binary_mask(ext_points,int_points,img.size)

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(cat_mask)

In [ ]:
##Get contours for image
ext_points = image['tags'][1]['segmentation'][9]['extPoints']
int_points = image['tags'][1]['segmentation'][9]['intPoints']

In [ ]:
##Format countours in right format
int_points = [list(map(tuple,int)) for int in int_points]

In [ ]:
##Format countours in right format
ext_points = list(map(tuple,ext_points))

In [ ]:
dog_mask = generate_binary_mask(ext_points,int_points,img.size)

In [ ]:
plt.imshow(dog_mask)

In [ ]:
final_mask = np.zeros_like(dog_mask)
indices = np.where(dog_mask == 1)
final_mask[indices] = 1
indices = np.where(cat_mask == 1)
final_mask[indices] = 2

In [ ]:
np.unique(final_mask)

In [ ]:
plt.imshow(final_mask)

In [ ]:
import seggpt_inference

In [ ]:
prompt_mask = final_mask
prompt_img = img

In [ ]:
req = requests.get("http://" + data["images"][0]["url"])
test_image = Image.open(BytesIO(req.content)).convert("RGB")

In [ ]:
out_mask,out_image = seggpt_inference.predict(prompt_img,prompt_mask,test_image,threshold = 80)

In [ ]:
plt.imshow(out_mask)

In [ ]:
plt.imshow(out_image)

In [ ]:
import numpy as np
from skimage.measure import label, regionprops

def separate_masks(binary_mask, area_threshold):
    # Label connected components in the binary mask
    labeled_mask = label(binary_mask)
    
    # Get region properties of each connected component
    regions = regionprops(labeled_mask)
    
    # Initialize an empty list to store individual masks
    separate_masks = []
    
    # Iterate over each region and create a separate binary mask
    for region in regions:
        # Filter regions based on area threshold
        if region.area >= area_threshold:
            instance_mask = (labeled_mask == region.label).astype(np.uint8)
            separate_masks.append(instance_mask)
    
    return separate_masks


In [ ]:
masks = separate_masks(out_mask,10)

In [ ]:
def combine_masks(masks):
    # Initialize an empty array to store the combined mask
    combined_mask = np.zeros_like(masks[0])

    # Assign unique instance IDs to each mask
    for i, mask in enumerate(masks, start=1):
        # Find the indices where the mask is True
        indices = np.where(mask == 1)

        # Assign the instance ID to those indices in the combined mask
        combined_mask[indices] = i

    return combined_mask

In [ ]:
combined_mask = combine_masks(masks)

In [ ]:
plt.imshow(combined_mask)

In [ ]:
plt.imshow(test_image)

In [ ]:
def show_segmentation_mask(mask, image):
    # Create a copy of the original image
    overlay = np.array(image.copy())

    # Apply the mask as a red overlay on the image
    red_overlay = np.zeros_like(overlay)
    red_overlay[..., 0] = mask * 255

    # Blend the red overlay with the original image
    blended = cv2.addWeighted(overlay, 0.7, red_overlay, 0.3, 0)
    return blended

In [ ]:
import numpy as np
import cv2

def overlay_segmentation(image, mask):
    # Convert the PIL image to a numpy array
    image_array = np.array(image)
    num_instances = len(np.unique(mask))
    # Create a copy of the image array to draw on
    overlay = image_array.copy()

    # Define colors for each instance ID
    colors = generate_equally_spaced_colors(num_instances)

    # Draw each instance in a different color on the overlay image
    for instance_id in np.unique(mask):
        if instance_id == 0:
            continue

        # Create a binary mask for the current instance ID
        instance_mask = np.where(mask == instance_id, 255, 0).astype(np.uint8)

        # Find contours in the binary mask
        contours, _ = cv2.findContours(instance_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

        # Draw the contours on the overlay image
        cv2.drawContours(overlay, contours, -1, colors[instance_id], thickness=cv2.FILLED)

    # Blend the overlay image with the original image
    blended_image = cv2.addWeighted(overlay, 0.5, image_array, 0.5, 0)

    # Convert the blended image back to PIL format
    blended_image_pil = Image.fromarray(blended_image)

    return blended_image_pil


In [ ]:
final_img = overlay_segmentation(test_image,combined_mask)

In [ ]:
final_img

In [ ]:
##Now lets auto-annotate the rest of the images using the single prompt_img/mask
for image in data['images']:
    if image['id'] == "64b7210228151f0007746978": ##Don't predict your prompt_image
        continue 
    req = requests.get("http://" + image["url"])
    test_image = Image.open(BytesIO(req.content)).convert("RGB")
    out_mask,out_image = seggpt_inference.predict(prompt_img,prompt_mask,test_image,threshold = 20)
    mask_overlay = show_segmentation_mask(out_mask,test_image)
    plt.imshow(test_image)
    plt.show()
    plt.imshow(out_image)
    plt.show()
    plt.imshow(out_mask)
    plt.show()
    plt.imshow(mask_overlay)
    plt.show()

In [ ]:
##Now lets auto-annotate the rest of the images using the single prompt_img/mask
for image in data['images']:
    if image['id'] == "64b7210228151f0007746978": ##Don't predict your prompt_image
        continue 
    req = requests.get("http://" + image["url"])
    test_image = Image.open(BytesIO(req.content)).convert("RGB")
    out_mask,out_image = seggpt_inference.predict(prompt_img,prompt_mask,test_image,threshold = 80)
    masks = separate_masks(out_mask,30)
    if len(masks) > 0:
        combined_mask = combine_masks(masks)
        mask_overlay = overlay_segmentation(test_image,combined_mask)
    else:
        mask_overlay = test_image
    plt.imshow(test_image)
    plt.show()
    plt.imshow(out_image)
    plt.show()
    plt.imshow(out_mask)
    plt.show()
    plt.imshow(mask_overlay)
    plt.show()

In [ ]:
test_image = Image.open("2023_Acura_TLX_EVOX.jpg").convert("RGB")

In [ ]:
out_mask,out_image = seggpt_inference.predict(prompt_img,prompt_mask,test_image,threshold = 120)
masks = separate_masks(out_mask,30)
if len(masks) > 0:
    combined_mask = combine_masks(masks)
    mask_overlay = overlay_segmentation(test_image,combined_mask)
else:
    mask_overlay = test_image

In [ ]:
plt.imshow(mask_overlay)